# Installing Prometheus

We will download the current stable version locally and just start the binary,  

In [4]:
WORKDIR="/tmp/lfs-prometheus"

PROM_VERSION="2.49.1"
DEMO_VERSION="0.11.1"
NODEEXP_VERSION="1.7.0"
CADVISOR_VERSION="0.36.0"

In [22]:
%%bash -s {WORKDIR}

{
    echo "Cloning the repo"
    WORKDIR=$1
    if [ ! -e ${WORKDIR}/.git ]; then 
        git clone https://github.com/ennc0d3-learn/lfs-prometheus ${WORKDIR} 
    else 
        cd ${WORKDIR} && git pull --rebase
        ls -l ${WORKDIR}
    fi
    echo "Ready to go!"
}

Cloning the repo


Cloning into '/tmp/lfs-prometheus'...


Ready to go!


In [23]:
%%bash -s  {WORKDIR} {PROM_VERSION}
(
    WORKDIR=$1
    PROM_VERSION=$2
    cd ${WORKDIR}
    wget -q https://github.com/prometheus/prometheus/releases/download/v${PROM_VERSION}/prometheus-${PROM_VERSION}.linux-amd64.tar.gz
    tar -zxvf prometheus-${PROM_VERSION}.*.gz
    
)


prometheus-2.49.1.linux-amd64/
prometheus-2.49.1.linux-amd64/NOTICE
prometheus-2.49.1.linux-amd64/LICENSE
prometheus-2.49.1.linux-amd64/prometheus
prometheus-2.49.1.linux-amd64/consoles/
prometheus-2.49.1.linux-amd64/consoles/prometheus-overview.html
prometheus-2.49.1.linux-amd64/consoles/node-cpu.html
prometheus-2.49.1.linux-amd64/consoles/node.html
prometheus-2.49.1.linux-amd64/consoles/prometheus.html
prometheus-2.49.1.linux-amd64/consoles/index.html.example
prometheus-2.49.1.linux-amd64/consoles/node-overview.html
prometheus-2.49.1.linux-amd64/consoles/node-disk.html
prometheus-2.49.1.linux-amd64/prometheus.yml
prometheus-2.49.1.linux-amd64/promtool
prometheus-2.49.1.linux-amd64/console_libraries/
prometheus-2.49.1.linux-amd64/console_libraries/menu.lib
prometheus-2.49.1.linux-amd64/console_libraries/prom.lib


In [24]:
%%bash -s {WORKDIR} {PROM_VERSION}
(
    WORKDIR=$1
    PROM_VERSION=$2
    prometheus_dir=${WORKDIR}/prometheus-${PROM_VERSION}*
    cat > ${prometheus_dir}/prometheus.yml <<-EOD
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
EOD
echo "Created the prometheus.yml"
cat ${prometheus_dir}/prometheus.yml
)

Created the prometheus.yml
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']


In [25]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    PROM_VERSION=$2
    cd $WORKDIR/prometheus-*/
    killall prometheus
    ./prometheus &
    
)

prometheus: no process found
ts=2024-02-15T10:53:07.998Z caller=main.go:544 level=info msg="No time or size retention was set so using the default time retention" duration=15d
ts=2024-02-15T10:53:08.002Z caller=main.go:588 level=info msg="Starting Prometheus Server" mode=server version="(version=2.49.1, branch=HEAD, revision=43e14844a33b65e2a396e3944272af8b3a494071)"
ts=2024-02-15T10:53:08.006Z caller=main.go:593 level=info build_context="(go=go1.21.6, platform=linux/amd64, user=root@6d5f4c649d25, date=20240115-16:58:43, tags=netgo,builtinassets,stringlabels)"
ts=2024-02-15T10:53:08.010Z caller=main.go:594 level=info host_details="(Linux 6.2.0-39-generic #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2 x86_64 ennc0d3 (none))"
ts=2024-02-15T10:53:08.010Z caller=main.go:595 level=info fd_limits="(soft=1048576, hard=1048576)"
ts=2024-02-15T10:53:08.010Z caller=main.go:596 level=info vm_limits="(soft=unlimited, hard=unlimited)"
ts=2024-02-15T10:53:08.024Z caller=web.go:565 

Process is interrupted.


Download the prepare the demo service 

In [27]:
%%bash -s {WORKDIR} {DEMO_VERSION}
(
    WORKDIR=$1
    DEMO_VERSION=${2:-0.11.1}
    cd $WORKDIR
    echo "Downloading the demo service version ${DEMO_VERSION}"
    wget -q https://github.com/juliusv/prometheus_demo_service/releases/download/${DEMO_VERSION}/prometheus_demo_service-${DEMO_VERSION}.linux-amd64
    
    chmod +x ./prometheus_demo_service-${DEMO_VERSION}.linux-amd64
    mv ./prometheus_demo_service-${DEMO_VERSION}.linux-amd64 ./prometheus_demo_service
    
    echo "Starting the demo service instances"
    
    killall ./prometheus_demo_service
    
    ./prometheus_demo_service -listen-address=":10001" & > /dev/null
    ./prometheus_demo_service -listen-address=":10002" & > /dev/null
    ./prometheus_demo_service -listen-address=":10003" & > /dev/null
    
    echo "Demo instances are running"
    
)

Starting the demo service instances


./prometheus_demo_service: no process found


Demo instances are running
Process is interrupted.


In [ ]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
EOD
    
    # Send signal to reaload
    killall -HUP prometheus
)



## Installing node exporter

In [28]:
%%bash -s {WORKDIR} {NODEEXP_VERSION}
{
    cleanup() {
        trap - INT TERM
        echo "Cleaning up the node_exporter"
        killall node_exporter
        exit 1
    }
    
    trap cleanup INT TERM
    
    WORKDIR=$1
    NODEEXP_VERSION=${2:-1.7.0}
    startNodeExporter() {
        cd $WORKDIR
        wget -q https://github.com/prometheus/node_exporter/releases/download/v${NODEEXP_VERSION}/node_exporter-${NODEEXP_VERSION}.linux-amd64.tar.gz -O node_exporter.tgz
        tar -zxf node_exporter.tgz
        chmod +x ./node_exporter*/node_exporter
        killall node_exporter
        ./node_exporter*/node_exporter & > /dev/null
    }
    echo "Download and start the exporter"
    startNodeExporter
}

Download and start the exporter


node_exporter: no process found
ts=2024-02-15T10:56:02.186Z caller=node_exporter.go:192 level=info msg="Starting node_exporter" version="(version=1.7.0, branch=HEAD, revision=7333465abf9efba81876303bb57e6fadb946041b)"
ts=2024-02-15T10:56:02.187Z caller=node_exporter.go:193 level=info msg="Build context" build_context="(go=go1.21.4, platform=linux/amd64, user=root@35918982f6d8, date=20231112-23:53:35, tags=netgo osusergo static_build)"
ts=2024-02-15T10:56:02.190Z caller=filesystem_common.go:111 level=info collector=filesystem msg="Parsed flag --collector.filesystem.mount-points-exclude" flag=^/(dev|proc|run/credentials/.+|sys|var/lib/docker/.+|var/lib/containers/storage/.+)($|/)
ts=2024-02-15T10:56:02.190Z caller=filesystem_common.go:113 level=info collector=filesystem msg="Parsed flag --collector.filesystem.fs-types-exclude" flag=^(autofs|binfmt_misc|bpf|cgroup2?|configfs|debugfs|devpts|devtmpfs|fusectl|hugetlbfs|iso9660|mqueue|nsfs|overlay|proc|procfs|pstore|rpc_pipefs|securityfs|seli

Process is interrupted.


Update the scrape targets to include the node_exporter service running on 9100

In [29]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
    - job_name: 'node_exporter'
      static_configs:
        - targets:
            - localhost:9100
EOD
    
    # Send signal to reaload
    killall -HUP prometheus
)


ts=2024-02-15T10:56:28.495Z caller=main.go:1245 level=info msg="Loading configuration file" filename=prometheus.yml
ts=2024-02-15T10:56:28.501Z caller=main.go:1282 level=info msg="Completed loading of configuration file" filename=prometheus.yml totalDuration=6.06281ms db_storage=4.213µs remote_storage=4.808µs web_handler=1.679µs query_engine=2.335µs scrape=279.895µs scrape_sd=120.948µs notify=6.819µs notify_sd=2.634µs rules=2.665µs tracing=9.039µs


Install/Run cAdvisor

In [31]:
%%bash -s {WORKDIR} {CADVISOR_VERSION}
{
    WORKDIR=$1
    CADVISOR_VERSION=${2:-"0.36.0"}
    
    cleanup() {
        trap - INT TERM
        echo "Stopping cAdvisor"
        docker rm -f cadvisor
        exit 1
    }
    
    startCadvisor() {
        docker rm -f cadvisor
        docker run \
        --volume=/:/rootfs:ro \
        --volume=/var/run:/var/run:ro \
        --volume=/sys:/sys:ro \
        --volume=/var/lib/docker/:/var/lib/docker:ro \
        --volume=/dev/disk/:/dev/disk:ro \
        --publish=8080:8080 \
        --detach=true \
        --name=cadvisor \
        --privileged \
        --device=/dev/kmsg \
        gcr.io/cadvisor/cadvisor:v${CADVISOR_VERSION}
    }
    
    trap cleanup INT TERM
    startCadvisor
}

cadvisor


0685fba780d42d6af7850eae5bcbd75248ae9a35f1a872330893df532c432c44


In [32]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
    - job_name: 'node_exporter'
      static_configs:
        - targets:
            - localhost:9100
    - job_name: 'cadvisor'
      static_configs:
        - targets:
            - localhost:8080
EOD
    
    # Send signal to reaload
    killall -HUP prometheus
)


ts=2024-02-15T10:57:02.796Z caller=main.go:1245 level=info msg="Loading configuration file" filename=prometheus.yml
ts=2024-02-15T10:57:02.797Z caller=main.go:1282 level=info msg="Completed loading of configuration file" filename=prometheus.yml totalDuration=1.074342ms db_storage=4.962µs remote_storage=6.255µs web_handler=2.786µs query_engine=2.21µs scrape=252.165µs scrape_sd=121.864µs notify=2.724µs notify_sd=3.866µs rules=2.903µs tracing=9.547µs


Node Exporter - Custom 

In this chapter, we create an simple Node Exporter (CPU metrics collection), the program is written in python [cpu-metrics-exporter.py](./cpu-metrics-exporter/cpu-metric-exporter.py)

It uses psutil and get the cpu data and use CounterMetricFamily to transform to prometheus format.

In [34]:
%%bash -s {WORKDIR}
{
    WORKDIR=$1
    cd $WORKDIR
    killall cpu-metric-exporter.py
    python cpu-metrics-exporter/cpu-metric-exporter.py
}


cpu-metric-exporter.py: no process found


python: can't open file '/tmp/lfs-prometheus/cpu-metrics-exporter/cpu-metric-exporter.py': [Errno 2] No such file or directory


CalledProcessError: Command 'b'{\n    WORKDIR=$1\n    cd $WORKDIR\n    killall cpu-metric-exporter.py\n    python cpu-metrics-exporter/cpu-metric-exporter.py\n}\n'' returned non-zero exit status 2.

In [35]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
    - job_name: 'node_exporter'
      static_configs:
        - targets:
            - localhost:9100
    - job_name: 'cadvisor'
      static_configs:
        - targets:
            - localhost:8080
    - job_name: 'cpu-metrics'
      static_configs:
        - targets:
            - localhost:8100
EOD
    # Send signal to reaload
    killall -HUP prometheus
)

ts=2024-02-15T10:57:49.562Z caller=main.go:1245 level=info msg="Loading configuration file" filename=prometheus.yml
ts=2024-02-15T10:57:49.563Z caller=main.go:1282 level=info msg="Completed loading of configuration file" filename=prometheus.yml totalDuration=750.152µs db_storage=2.658µs remote_storage=2.961µs web_handler=625ns query_engine=1.305µs scrape=137.695µs scrape_sd=88.88µs notify=1.055µs notify_sd=1.876µs rules=1.676µs tracing=6.259µs
